# Collect Data (Game 7)

In [6]:
import requests
import pandas as pd
import numpy as np

# NHL API:https://github.com/dword4/nhlapi
# https://www.dataquest.io/blog/python-api-tutorial/
# https://stackoverflow.com/questions/41100303/convert-api-to-pandas-dataframe
r = requests.get('https://statsapi.web.nhl.com/api/v1/game/2018030417/feed/live')

# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html
# pandas data frame of allPlays
x = r.json()

# Allplays DataFrame
df = pd.DataFrame(x['liveData']['plays']['allPlays'])
df

# Creating variable for normalizing nested data
data = x['liveData']['plays']['allPlays']
data

# Flatten allPlays nested data
results_df = pd.json_normalize(data)
results_df

# Define & Flatten Game Data
gamedata_df = pd.DataFrame(x['gameData'])
game_num = gamedata_df['game'][0]

# Define & Flatten Coordinates data
coords = df.iloc[:,2]
coords_df = pd.json_normalize(coords)
coords_df

# Define & Flatten Events data
events = df.iloc[:,0][3:]
event_df = pd.json_normalize(events)
event_df

# Define & Flatten Period data
period = df.iloc[:,1][2:]
period_df = pd.json_normalize(period)
period_df['period']

# Define & Flatten Time data
time = df.iloc[:,1][2:]
time_df = pd.json_normalize(time)
time_df['periodTime']
time_df['dateTime']

# Define & Flatten Date data
date = time_df['dateTime'][:9].str[:10]

# Define & Flatten Home/Away Team data
home_away = gamedata_df.iloc[[10,11]]
home_away_df = pd.json_normalize(home_away.teams.to_dict())
home_away_df['away.name']
home_away_df['home.name']

# create empty array for putting event coordinate data in
latitude = []
longitude = []

# iterative loop for pushing data into arrays
for i in coords:

    if coords_df['x'] is not None:

        lat = coords_df['y']
        lon = coords_df['x']
        for i in [lat]:
            latitude.append(i)
        for i in [lon]:
            longitude.append(i)
      
# Creates datafrome with events that includes coordinates
df = pd.DataFrame(list
                  (zip(results_df['result.event'],
                       results_df['result.description'],
                       results_df['team.name'],
                       results_df['team.triCode'],
                       period_df['period'],
                       time_df['periodTime'],
                       time_df['dateTime'].str[:10],
                       lat,
                       lon,
                       results_df['result.strength.name'])), 
                  columns = ['Event',
                             'Description',
                             'Team',
                             'Tricode',
                             'Period',
                             'Time',
                             'Date',
                             'Lat',
                             'Lon',
                             'Strength'])

# Removes rows with "NaN" stoppages and period ends (https://stackoverflow.com/questions/39339935/pandas-dropping-rows-with-missing-data-not-working-using-isnull-notnull)
# df.replace(["NaN", 'NaT'], np.nan, inplace = True)
# df = df.dropna()

# Create a column comparing tricode of Dataframe to tricode of API and tell me which team is home/away
df['Home_Away_Team'] = np.where(df['Tricode'].values == home_away_df['home.triCode'].values, 'Away', 'Home')

# Create new column that returns opposite longitude for everything during 2nd Period (*-1)
# df['Lon_2ndPer'] = np.where(df['Period'] == 2, df['Lon']*-1, df['Lon'])
# Transpose the New column over the old "Lon"
df['Lon'] = np.where(df['Period'] == 2, df['Lon']*-1, df['Lon'])

pd.set_option("display.max_rows",30)

df

,Event,Description,Team,Tricode,Period,Time,Date,Lat,Lon,Strength,Home_Away_Team
0,Game Scheduled,Game Scheduled,NaN,NaN,1,00:00,2019-06-13,NaN,NaN,NaN,Home
1,Period Ready,Period Ready,NaN,NaN,1,00:00,2019-06-13,NaN,NaN,NaN,Home
2,Period Start,Period Start,NaN,NaN,1,00:06,2019-06-13,NaN,NaN,NaN,Home
3,Faceoff,Sean Kuraly faceoff won against Oskar Sundqvist,Boston Bruins,BOS,1,00:10,2019-06-13,0.0,0.0,NaN,Away
4,Hit,Ivan Barbashev hit Charlie McAvoy,St. Louis Blues,STL,1,00:10,2019-06-13,-35.0,-65.0,NaN,Home
...,...,...,...,...,...,...,...,...,...,...,...
294,Stoppage,Offside,NaN,NaN,3,19:25,2019-06-13,NaN,NaN,NaN,Home
295,Faceoff,David Krejci faceoff won against Brayden Schenn,Boston Bruins,BOS,3,19:27,2019-06-13,22.0,20.0,NaN,Away
296,Hit,Torey Krug hit Alexander Steen,Boston Bruins,BOS,3,20:00,2019-06-13,38.0,48.0,NaN,Away
297,Missed Shot,Danton Heinen - Wide of Net,Boston Bruins,BOS,3,20:00,2019-06-13,-27.0,52.0,NaN,Away


# All Events & Filters (Figure 1)

In [7]:
import plotly.graph_objects as go

# Use Mapbox Token
mapbox_access_token = open("mapbox_token.txt").read() # Create a local file with mapbox token

# Home Team Events
home = df[df['Home_Away_Team'].str.contains('Home')]
home_lat = df[df['Home_Away_Team'].str.contains('Home')]['Lat']
home_lon = df[df['Home_Away_Team'].str.contains('Home')]['Lon']
home_text = df[df['Home_Away_Team'].str.contains('Home')]['Description']

# Away Team Events
away = df[df['Home_Away_Team'].str.contains('Away')]
away_lat = df[df['Home_Away_Team'].str.contains('Away')]['Lat']
away_lon = df[df['Home_Away_Team'].str.contains('Away')]['Lon']
away_text = df[df['Home_Away_Team'].str.contains('Away')]['Description']

# Display Data for Goals
goal = df[df['Event'].str.contains('Goal')]
goal_lat = df[df['Event'].str.contains('Goal')]['Lat']
goal_lon = df[df['Event'].str.contains('Goal')]['Lon']
goal_text = df[df['Event'].str.contains('Goal')]['Description']

# Display Data for Hits
hits = df[df['Event'].str.contains('Hit')]
hits_lat = df[df['Event'].str.contains('Hit')]['Lat']
hits_lon = df[df['Event'].str.contains('Hit')]['Lon']
hits_text = df[df['Event'].str.contains('Hit')]['Description']

# Display Data for Faceoffs
faceoff = df[df['Event'].str.contains('Faceoff')]
faceoff_lat = df[df['Event'].str.contains('Faceoff')]['Lat']
faceoff_lon = df[df['Event'].str.contains('Faceoff')]['Lon']
faceoff_text = df[df['Event'].str.contains('Faceoff')]['Description']

# Display Data for Shots
shots = df[df['Event'].str.match('Shot')]
shots_lat = df[df['Event'].str.match('Shot')]['Lat']
shots_lon = df[df['Event'].str.match('Shot')]['Lon']
shots_text = df[df['Event'].str.match('Shot')]['Description']

# Display Data for Blocked Shots
bl_shots = df[df['Event'].str.contains('Blocked Shot')]
bl_shots_lat = df[df['Event'].str.contains('Blocked Shot')]['Lat']
bl_shots_lon = df[df['Event'].str.contains('Blocked Shot')]['Lon']
bl_shots_text = df[df['Event'].str.contains('Blocked Shot')]['Description']

# Display Data for Missed Shots
m_shots = df[df['Event'].str.contains('Missed Shot')]
m_shots_lat = df[df['Event'].str.contains('Missed Shot')]['Lat']
m_shots_lon = df[df['Event'].str.contains('Missed Shot')]['Lon']
m_shots_text = df[df['Event'].str.contains('Missed Shot')]['Description']

# Display Data for Penalties rgb(0, 255, 0)
penalties = df[df['Event'].str.contains('Penalty')]
pen_lat = df[df['Event'].str.contains('Penalty')]['Lat']
pen_lon = df[df['Event'].str.contains('Penalty')]['Lon']
pen_text = df[df['Event'].str.contains('Penalty')]['Description']

# Display Data for Giveaways rgb(51, 0, 0)
giveaway = df[df['Event'].str.contains('Giveaway')]
give_lat = df[df['Event'].str.contains('Giveaway')]['Lat']
give_lon = df[df['Event'].str.contains('Giveaway')]['Lon']
give_text = df[df['Event'].str.contains('Giveaway')]['Description']

# Display Data for Takeaways rgb(51, 0, 0)
takeaway = df[df['Event'].str.contains('Takeaway')]
take_lat = df[df['Event'].str.contains('Takeaway')]['Lat']
take_lon = df[df['Event'].str.contains('Takeaway')]['Lon']
take_text = df[df['Event'].str.contains('Takeaway')]['Description']

fig1 = go.Figure()
# Home Event Markers
fig1.add_trace(go.Scattermapbox(
        lat = home_lat,
        lon = home_lon,
        mode = 'markers',
        marker = go.scattermapbox.Marker(
            size = 20,
            color = '#0000ff',
            opacity = 0.25
        ),
        text = home_text,
        hoverinfo = 'text',
        name = "Home",
#         https://plotly.com/python/hover-text-and-formatting/
        customdata = home,
        hovertemplate =
        '<b>Period</b>: %{customdata[4]}<br>'+
        '<b>Time</b>: %{customdata[5]}<br>'+
        '<b>Event</b>: %{customdata[0]}<br>'+
        '<b>Description</b>: %{customdata[1]}<br>'+
        '<b>Location</b>: %{customdata[10]}',
    ))

# Away Event Markers
fig1.add_trace(go.Scattermapbox(
        lat = away_lat,
        lon = away_lon,
        mode = 'markers',
        marker = go.scattermapbox.Marker(
            size = 20,
            color = 'rgb(255, 0, 0)',
            opacity = 0.25
        ),
        text = away_text,
        hoverinfo = 'text',
        name = "Away",
#         https://plotly.com/python/hover-text-and-formatting/
        customdata = away,
        hovertemplate =
        '<b>Period</b>: %{customdata[4]}<br>'+
        '<b>Time</b>: %{customdata[5]}<br>'+
        '<b>Event</b>: %{customdata[0]}<br>'+
        '<b>Description</b>: %{customdata[1]}<br>'+
        '<b>Location</b>: %{customdata[10]}',
    ))
# Goal Event Markers
fig1.add_trace(go.Scattermapbox(
        lat = goal_lat,
        lon = goal_lon,
        mode = 'markers',
        marker = go.scattermapbox.Marker(
            size = 18,
            color = '#FFA500',
            opacity = 0.75
        ),
        text = goal_text,
        hoverinfo = 'text',
        name = "Goal",
#         https://plotly.com/python/hover-text-and-formatting/
        customdata = goal,
        hovertemplate =
        '<b>Period</b>: %{customdata[4]}<br>'+
        '<b>Time</b>: %{customdata[5]}<br>'+
        '<b>Event</b>: %{customdata[0]}<br>'+
        '<b>Description</b>: %{customdata[1]}<br>'+
        '<b>Location</b>: %{customdata[10]}<br>'+
        '<b>Strength</b>: %{customdata[9]}',
    ))

# Hits Event Markers
fig1.add_trace(go.Scattermapbox(
        lat = hits_lat,
        lon = hits_lon,
        mode = 'markers',
        marker = go.scattermapbox.Marker(
            size = 14,
            color = 'rgb(255,255,0)',
            opacity=0.7
        ),
        text=hits_text,
        hoverinfo='text',
        name="Hit",
#         https://plotly.com/python/hover-text-and-formatting/
        customdata=hits,
        hovertemplate =
        '<b>Period</b>: %{customdata[4]}<br>'+
        '<b>Time</b>: %{customdata[5]}<br>'+
        '<b>Event</b>: %{customdata[0]}<br>'+
        '<b>Description</b>: %{customdata[1]}<br>'+
        '<b>Location</b>: %{customdata[10]}',
    ))

# Faceoff Event Markers
fig1.add_trace(go.Scattermapbox(
        lat=faceoff_lat,
        lon=faceoff_lon,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=8,
            color='rgb(255,165,0)',
            opacity=0.10
        ),
        text=faceoff_text,
        hoverinfo='text',
        name="Faceoff",
#         https://plotly.com/python/hover-text-and-formatting/
        customdata=faceoff,
        hovertemplate =
        '<b>Period</b>: %{customdata[4]}<br>'+
        '<b>Time</b>: %{customdata[5]}<br>'+
        '<b>Event</b>: %{customdata[0]}<br>'+
        '<b>Description</b>: %{customdata[1]}<br>'+
        '<b>Location</b>: %{customdata[10]}',
    ))

# Shots Event Markers
fig1.add_trace(go.Scattermapbox(
        lat=shots_lat,
        lon=shots_lon,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=16,
            color='#008000',
            opacity=0.30
        ),
        text=shots_text,
        hoverinfo='text',
        name="Shot",
#         https://plotly.com/python/hover-text-and-formatting/
        customdata=shots,
        hovertemplate =
        '<b>Period</b>: %{customdata[4]}<br>'+
        '<b>Time</b>: %{customdata[5]}<br>'+
        '<b>Event</b>: %{customdata[0]}<br>'+
        '<b>Description</b>: %{customdata[1]}<br>'+
        '<b>Location</b>: %{customdata[10]}',
    ))

# Missed Shots Event Markers
fig1.add_trace(go.Scattermapbox(
        lat=m_shots_lat,
        lon=m_shots_lon,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=10,
            color='#008000',
            opacity=1
        ),
        text=m_shots_text,
        hoverinfo='text',
        name="Missed Shot",
#         https://plotly.com/python/hover-text-and-formatting/
        customdata=m_shots,
        hovertemplate =
        '<b>Period</b>: %{customdata[4]}<br>'+
        '<b>Time</b>: %{customdata[5]}<br>'+
        '<b>Event</b>: %{customdata[0]}<br>'+
        '<b>Description</b>: %{customdata[1]}<br>'+
        '<b>Location</b>: %{customdata[10]}',
    ))

# Blocked Shots Event Markers
fig1.add_trace(go.Scattermapbox(
        lat=bl_shots_lat,
        lon=bl_shots_lon,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=10,
            color='#008000',
            opacity=0.5
        ),
        text=bl_shots_text,
        hoverinfo='text',
        name="Blocked Shot",
#         https://plotly.com/python/hover-text-and-formatting/
        customdata=bl_shots,
        hovertemplate =
        '<b>Period</b>: %{customdata[4]}<br>'+
        '<b>Time</b>: %{customdata[5]}<br>'+
        '<b>Event</b>: %{customdata[0]}<br>'+
        '<b>Description</b>: %{customdata[1]}<br>'+
        '<b>Location</b>: %{customdata[10]}',
    ))

# Penalty Event Markers
fig1.add_trace(go.Scattermapbox(
        lat=pen_lat,
        lon=pen_lon,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=10,
            color='rgb(0, 255, 000)',
            opacity=0.7
        ),
        text=pen_text,
        hoverinfo='text',
        name="Penalty",
#         https://plotly.com/python/hover-text-and-formatting/
        customdata=penalties,
        hovertemplate =
        '<b>Period</b>: %{customdata[4]}<br>'+
        '<b>Time</b>: %{customdata[5]}<br>'+
        '<b>Event</b>: %{customdata[0]}<br>'+
        '<b>Description</b>: %{customdata[1]}<br>'+
        '<b>Location</b>: %{customdata[10]}',
    ))

# Giveaway Event Markers
fig1.add_trace(go.Scattermapbox(
        lat=give_lat,
        lon=give_lon,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=15,
            color='#FF0000',
            opacity=0.5,
            symbol = 'circle'
        ),
        text=give_text,
        hoverinfo='text',
        name="Giveaway",
    #         https://plotly.com/python/hover-text-and-formatting/
        customdata=giveaway,
        hovertemplate =
        '<b>Period</b>: %{customdata[4]}<br>'+
        '<b>Time</b>: %{customdata[5]}<br>'+
        '<b>Event</b>: %{customdata[0]}<br>'+
        '<b>Description</b>: %{customdata[1]}<br>'+
        '<b>Location</b>: %{customdata[10]}',
    ))

# Takeaway Event Markers
fig1.add_trace(go.Scattermapbox(
        lat=take_lat,
        lon=take_lon,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=15,
            color='#FF0000',
            opacity=0.5,
            symbol = 'circle',
        ),
        text=take_text,
        hoverinfo='text',
        name="Takeaway",
    #         https://plotly.com/python/hover-text-and-formatting/
        customdata=takeaway,
        hovertemplate =
        '<b>Period</b>: %{customdata[4]}<br>'+
        '<b>Time</b>: %{customdata[5]}<br>'+
        '<b>Event</b>: %{customdata[0]}<br>'+
        '<b>Description</b>: %{customdata[1]}<br>'+
        '<b>Location</b>: %{customdata[10]}',
    ))

#Legend
fig1.update_layout(
    width=800,
    height=500,
    margin={"r":10,"t":40,"l":10,"b":20},
    title='2019 Stanley Cup Finals (Game 7)',
    autosize=False,
    hovermode='closest',
    hoverlabel=dict(
        bgcolor="white", 
        font_size=16, 
        font_family="Rockwell"
    ),
    showlegend=True,
#     legend_title='<b>Events</b>',
    legend_orientation="v",
        legend=dict(
        x=-.25,
        y=.875,
        traceorder="normal",
        font=dict(
            family="Rockwell",
            size=18,
            color="black"
        ),
        bgcolor="white",
        bordercolor="Black",
        borderwidth=2
    ),
    mapbox=dict(
        accesstoken=mapbox_access_token,
        center=dict(
            lat=0,
            lon=0
        ),
        pitch=0,
        zoom=1,
        bearing=0,
        style='mapbox://styles/isaacv/ckar8gbe708uw1ilmmvzdflsg'
    ),
)

fig1.show()

# Event Sequence Leading to Goal (Figure 2)

In [8]:
import plotly.graph_objects as go

# Use Mapbox Token
mapbox_access_token = open("mapbox_token.txt").read() # Create a local file with mapbox token

# prints a data frame of the first goal and events leading up to, begins w/ f/o
a_seq = df[(df['Time'] > '15:33') & (df['Time'] < '17:15') & (df['Period'] == 1) & (df['Event'] != 'Hit')]
a_seq

# prints a data frame of the second goal and events leading up to, begins w/ f/o
b_seq = df[(df['Time'] > '19:19') & (df['Time'] < '19:57') & (df['Period'] == 1) & (df['Event'] != 'Hit')]
b_seq

# prints a data frame of the third goal and events leading up to, begins w/ f/o
c_seq = df[(df['Time'] > '08:03') & (df['Time'] < '12:12') & (df['Period'] == 3) & (df['Lat'] != 0) & (df['Lon'] != 0) & (df['Event'] != 'Hit')]
c_seq

# prints a data frame of the fourth goal and events leading up to, begins w/ f/o
d_seq = df[(df['Time'] > '14:37') & (df['Time'] < '15:40') & (df['Period'] == 3) & (df['Event'] != 'Hit')]
d_seq

# prints a data frame of the fifth goal and events leading up to, begins w/ f/o
e_seq = df[(df['Time'] > '17:45') & (df['Time'] < '18:32') & (df['Period'] == 3) & (df['Lat'] != 0) & (df['Lon'] != 0) & (df['Event'] != 'Hit')] 
e_seq

# Display Data for Goals
goal = df[df['Event'].str.contains('Goal')]
goal_lat = df[df['Event'].str.contains('Goal')]['Lat']
goal_lon = df[df['Event'].str.contains('Goal')]['Lon']
goal_text = df[df['Event'].str.contains('Goal')]['Description']

fig2 = go.Figure()

# 1st Goal
fig2 = go.Figure(go.Scattermapbox(
    mode = "markers+lines",
    lon = a_seq['Lon'],
    lat = a_seq['Lat'],
    marker = {'size': 10},
    customdata=a_seq,
    name = "1st Goal (STL)",
    hovertemplate =
        '<b>Period</b>: %{customdata[4]}<br>'+
        '<b>Time</b>: %{customdata[5]}<br>'+
        '<b>Event</b>: %{customdata[0]}<br>'+
        '<b>Description</b>: %{customdata[1]}<br>'+
        '<b>Location</b>: %{customdata[10]}',
))

# 2nd Goal
fig2.add_trace(go.Scattermapbox(
    mode = "markers+lines",
    lon = b_seq['Lon'],
    lat = b_seq['Lat'],
    marker = {'size': 10},
    customdata=b_seq,
    name = "2nd Goal (STL)",
    hovertemplate =
        '<b>Period</b>: %{customdata[4]}<br>'+
        '<b>Time</b>: %{customdata[5]}<br>'+
        '<b>Event</b>: %{customdata[0]}<br>'+
        '<b>Description</b>: %{customdata[1]}<br>'+
        '<b>Location</b>: %{customdata[10]}',
))

# 3rd Goal
fig2.add_trace(go.Scattermapbox(
    mode = "markers+lines",
    lon = c_seq['Lon'],
    lat = c_seq['Lat'],
    marker = {'size': 10},
    customdata=c_seq,
    name = "3rd Goal (STL)",
    hovertemplate =
        '<b>Period</b>: %{customdata[4]}<br>'+
        '<b>Time</b>: %{customdata[5]}<br>'+
        '<b>Event</b>: %{customdata[0]}<br>'+
        '<b>Description</b>: %{customdata[1]}<br>'+
        '<b>Location</b>: %{customdata[10]}',
))

# 4th Goal
fig2.add_trace(go.Scattermapbox(
    mode = "markers+lines",
    lon = d_seq['Lon'],
    lat = d_seq['Lat'],
    marker = {'size': 10},
    customdata=d_seq,
    name = "4th Goal (STL)",
    hovertemplate =
        '<b>Period</b>: %{customdata[4]}<br>'+
        '<b>Time</b>: %{customdata[5]}<br>'+
        '<b>Event</b>: %{customdata[0]}<br>'+
        '<b>Description</b>: %{customdata[1]}<br>'+
        '<b>Location</b>: %{customdata[10]}',
))

# 5th Goal
fig2.add_trace(go.Scattermapbox(
    mode = "markers+lines",
    lon = e_seq['Lon'],
    lat = e_seq['Lat'],
    marker = go.scattermapbox.Marker(
            size = 12,
            color = 'rgb(255,255,0)',
            opacity = 0.75
        ),
    customdata=e_seq,
    name = "5th Goal (BOS)",
    hovertemplate =
        '<b>Period</b>: %{customdata[4]}<br>'+
        '<b>Time</b>: %{customdata[5]}<br>'+
        '<b>Event</b>: %{customdata[0]}<br>'+
        '<b>Description</b>: %{customdata[1]}<br>'+
        '<b>Location</b>: %{customdata[10]}',
))

# Goal Event Markers
fig2.add_trace(go.Scattermapbox(
        lat = goal_lat,
        lon = goal_lon,
        mode = 'markers',
        marker = go.scattermapbox.Marker(
            size = 18,
            color = '#FFA500',
            opacity = 0.75
        ),
        text = goal_text,
        hoverinfo = 'text',
        name = "Goal(s)",
#         https://plotly.com/python/hover-text-and-formatting/
        customdata = goal,
        hovertemplate =
        '<b>Period</b>: %{customdata[4]}<br>'+
        '<b>Time</b>: %{customdata[5]}<br>'+
        '<b>Event</b>: %{customdata[0]}<br>'+
        '<b>Description</b>: %{customdata[1]}<br>'+
        '<b>Location</b>: %{customdata[10]}<br>'+
        '<b>Location</b>: %{customdata[9]}',
    ))

#Legend
fig2.update_layout(
    margin ={'l':0,'t':25,'b':0,'r':0},
    width=800,
    height=500,
    title='Event Sequence Leading to Goal (Game 7)',
    showlegend=True,
#     legend_title='<b>Events</b>',
    legend_orientation="v",
        legend=dict(
        x=-.25,
        y=.875,
        traceorder="normal",
        font=dict(
            family="Rockwell",
            size=18,
            color="black"
        ),
        bgcolor="white",
        bordercolor="Black",
        borderwidth=2
    ),
    mapbox=dict(
        accesstoken=mapbox_access_token,
        center=dict(
            lat=0,
            lon=0
        ),
        pitch=0,
        zoom=1,
        bearing=0,
        style='mapbox://styles/isaacv/ckar8gbe708uw1ilmmvzdflsg'
    ),
)

fig2.show()